In [34]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# OOP

## Recap

  - Encapsulation: grouping data and methods that work on that data. 
    - When initiating an instance, some data including parameters are passed to the constructor. The output of the constructor is an object that has some data and methods that work on that data.
  - Instance: an object that is created from a class -- a model under a special set of paramters values (values are given). 
    - An instance has attributes and methods -- instance attributes (properties) and instance methods.
  - Composition: a class's instance can have instances of other classes as attributes. `def __init__(self, other_class_instance): self.other_class_instance = other_class_instance`.
    - A class can have instance attributes that are instances of other classes. 
    - A class can have instance methods that use instances of other classes as parameters.

In [1]:
class Consumer_LogUtility:
    def __init__(self, beta):
        self.beta = beta
    def foc_c(self, knext: float, w: float, R: float): # (23.6b)
        """
        First order condition of the consumer's problem
        (knext, w, R)
        """
        return knext - (self.beta/(1+self.beta))*w

class Producer_cobbDouglas:
    def __init__(self, alpha):
        self.alpha = alpha
    def foc_l(self, k, w, l=1): # (23.9)
        """ 
        FOC with respect to labor
        (k, l, w)
        """
        return (1-self.alpha)*(k/l)**self.alpha - w
    def foc_k(self, k, R, l=1): # (23.9)
        """
        FOC with respect to capital
        (k, l, R)
        """
        return self.alpha*(l/k)**(1-self.alpha) - R


In [ ]:
class OLGMarket:
    def __init__(self, consumer, producer, k0):
        self.consumer = consumer # composite 1
        self.producer = producer # composite 2
        self.k0 = k0
        self.k = k0
        self.equilibrium_dynamics = [[k0, None, None]]
    def generate_onePeriod_equilibrium(self):
        """
        return a tuple of (knext, w, R)
        and track the dynamics in self.equilibrium_dynamics
        """ # docstring: documentation string
        
        alpha = self.producer.alpha
        beta = self.consumer.beta

        w = (1-alpha)*self.k**alpha
        R = alpha*(self.k)**(alpha-1)
        knext = (beta/(1+beta))*w

        # keep track of the dynamics
        self.equilibrium_dynamics[len(self.equilibrium_dynamics)-1][1] = w
        self.equilibrium_dynamics[len(self.equilibrium_dynamics)-1][2] = R
        self.equilibrium_dynamics.append([knext, None, None])

        # set the market time to the next period
        self.k = knext
        return (knext, w, R)
    
    def clear_equilibrium_dynamics(self):
        self.equilibrium_dynamics = [[self.k0, None, None]]
        self.k = self.k0



## Design a prototype

Designing a class all at once is daunting. We can start with only the properties (attributes) of the class and then add methods as we go, i.e. only setup a prototype with only `def __init__(...)`.


Consider the following AR(k) model:

$$
y_t = \phi_1 y_{t-1} + \phi_2 y_{t-2} + \cdots + \phi_k y_{t-k} + \epsilon_t
$$

where $\epsilon_t \sim N(0, \sigma^2)$.

Suppose we are interested in simulate a random path of $y_t$ for $t=1,2,\ldots,T$, with $Y_0=[y_0, y_{-1}, \ldots, y_{-k+1}]$ given.

In [128]:
import numpy as np

class AR:
    def __init__(self, *phi, epsilon, Y0):
        self.phi = np.array(phi)
        self.epsilon = epsilon
        self.Y0 = np.array(Y0)
        self.memory = np.array([])
        self.YPast = np.array(self.Y0)

- We will talk about `*phi` later.
- epsilon is a function to generate random numbers. (An example will follow.)
- We start from `t=1`, for `y_1` we need `y_0` and `y_{t-k}` to kick start our simulation. `Y0` is a list of $y_0, y_{-1}, \ldots, y_{-k+1}$.  
- `memory` is to memorize simulated process so far (detail will follow).
- `YPast` is refering to $y_{t-1}, y_{t-2}, \ldots, y_{t-k}$ for a given `t>0`.

Consider the following random process:
$$
\begin{aligned}
y_t =0.8 y_{t-1} - 0.35 y_{t-2}  + \epsilon_t \\
Y_0= [0, 0]\\
\epsilon_t \sim N(0, 0.4^2) \\
\end{aligned}
$$

with $y_{0}=y_{-1}=0$

The above process fundamentals are determined by:

- `phi = [0.8, -0.35]`
- `Y0 = [0, 0]`, and
- `epsilon` that can generate random numbers from $N(0, 0.4^2)$.

### An instance

In [ ]:
import numpy as np

def Epsilon(mu, sigma):
    def draw(size):
        return np.random.normal(mu, sigma, size)
    return draw
    
epsilon = Epsilon(0, 0.4)

In [134]:
epsilon(1)
epsilon(10)

array([0.1190284])

array([-0.19489332,  0.26983268,  0.44204879,  0.20951998, -0.09668644,
       -0.2569766 , -0.40127042, -0.93600622,  0.25254926, -0.62819101])

In [135]:
ar = AR(0.8, -0.35, epsilon=epsilon, Y0=[0,0])

In [140]:
# an instance of the basic prototype of a class
ar.phi
ar.epsilon(10)
ar.Y0
ar.YPast

array([ 0.8 , -0.35])

array([ 0.12668316, -0.13900703,  0.02054825, -0.21399267, -0.15868743,
        0.1387678 , -0.05865975,  0.26106272,  0.20111539, -0.19960375])

array([0, 0])

array([0, 0])

## Packing

Packing is to pack all non-key-value-pair argument values into a tuple. 

In [2]:
def test(*args, epsilon):
    print(args)
    print(epsilon)

test(1,2,3, epsilon="Hi")

(1, 2, 3)
Hi


> In `test(1,2,3, epsilon="Hi")`: `1,2,3` are non-key-value inputs, and `epsilon="Hi"` is a key-value input. `1,2,3` will be packed into `args` tuple.

> In AR class, `*phi` will pack all non-key-value-pair argument values into `phi` tuple.

### Exercise CES and packing

In general a CES utility function of $k$ goods can be expressed as

$$
u(x_1, x_2, \ldots, x_k) = \left(\sum_{i=1}^k \alpha_i x_i^\rho\right)^{1/\rho}
$$

where $\alpha_i$ is the share parameter of good $i$ and $\rho$ is the elasticity of substitution. Consider the following two examples:

1. $(\alpha_1, \alpha_2, \alpha_3)=(0.3, 0.3, 0.4)$ and $\rho=2$. Compute the util under $\{x_1, x_2, x_3\}=\{2, 4, 5\}$.
2. $(\alpha_1, \alpha_2, \alpha_3, \alpha_4)=(0.2, 0.2, 0.3, 0.3)$ and $\rho=2$. Compute the util under $\{x_1, x_2, x_3, x_4\}=\{1, 2, 2, 1\}$.
   
Construct `CESUtility` function so that these two examples can be computed as:

In [ ]:
# example 1
u = CESUtility(0.3, 0.3, 0.4, rho=2)
u(2, 4, 5)

# example 2
u2 = CESUtility(0.2, 0.2, 0.3, 0.3, rho=2)
u(1, 2, 2, 1)


## Memory

Sometimes we want to an instance to memorize the method calls that have been made on it -- memorize the information that the method call has generated under the hood.

We can use 

## Design instance methods

To design an instance method, think what you want to do with the instance's attributes (properties). In our case, is to complete your task using **only `ar` instance**. 

> Using only `ar` is the key of OOP encapsulation feature. If you need to use information other than `ar`, you should pass them as parameters in the method or as instance attributes when you build `__init__` content.
>
> Sometimes we refer to the parameters that pass to `__init__` as **model parameters** (the fundamental description of your model setup) or and the parameters that pass to a method as **method parameters**.

Suppose you want to design a method so that `ar.simulate_nPeriods(10)` will simulate 10 periods of the AR(k) model.

Let's start from simulate only 1 period:

Algorithm:

0. Given `eps = ar.epsilon(1)`
1. Compute `Y_one_step_ahead` using `ar.phi` and `ar.YPast`, and `ar.epsilon`.
    - `ar.phi@ar.YPast + ar.epsilon(1)`
2. Update `YPast` by appending `Y_one_step_ahead` to `YPast` front and removing the last element of `YPast`.
3. Update `ar.memroy` to remember `Y_one_step_ahead`.

In [53]:
eps = ar.epsilon(1)

# Simulate one period of ar process
y_onePeriod_ahead = ar.phi@ar.YPast + eps

# Update YPast
ar.YPast = np.append(y_onePeriod_ahead, ar.YPast[:-1])

## append the new element to the memory
ar.memory = np.append(ar.memory, y_onePeriod_ahead)

> In `Update YPast` step, we need to re-bind new past values back to `YPast` since the RHS operation is not a change under the hood operation. To keep the change, we need to re-bind the new values back to `YPast`.

In [55]:
ar.YPast


array([-0.86352238, -0.59638357])

We can wrap up the above algorithm into a function that takes the `ar` instance as its **FIRST** input.

> Instance must be its first input for later use as a method.

In [56]:
def simulate_onePeriod(ar, eps):
    # Simulate one period of ar process
    y_onePeriod_ahead = ar.phi@ar.YPast + eps

    # Update YPast
    ar.YPast = np.append(y_onePeriod_ahead, ar.YPast[:-1])

    ## append the new element to the memory
    ar.memory = np.append(ar.memory, y_onePeriod_ahead)

> Notice the function can change the instance attributes under the hood.

### Test your helper function

In [57]:
ar = AR(0.8, -0.35, epsilon=epsilon, Y0=[0,0])
ar.memory

array([], dtype=float64)

In [63]:
simulate_onePeriod(ar, ar.epsilon(1))
ar.memory

array([-1.01842027, -0.8084974 , -0.13720271,  0.67056381,  0.67456232])

We can generalize it into n period simulation:

In [64]:
# generate n periods of ar process
n=10
eps = ar.epsilon(n)
for i in range(n):
    simulate_onePeriod(ar, eps[i])

In [65]:
ar.memory

array([-1.01842027, -0.8084974 , -0.13720271,  0.67056381,  0.67456232,
       -0.47305355, -0.26239135, -0.18650152,  0.19738215,  0.2394655 ,
       -0.04593107, -0.51002623, -0.45681947, -0.88574869, -0.49117195])

### Helper functions

In [66]:
def simulate_onePeriod(ar, eps):
    # Simulate one period of ar process
    y_onePeriod_ahead = ar.phi@ar.YPast + eps

    # Update YPast
    ar.YPast = np.append(y_onePeriod_ahead, ar.YPast[:-1])

    ## append the new element to the memory
    ar.memory = np.append(ar.memory, y_onePeriod_ahead)

def simulate_nPeriods(ar, n):
    eps = ar.epsilon(n)
    for i in range(n):
        simulate_onePeriod(ar)


In summary, to design your instance method:

1. create an instance.
2. use only the instance to lay out your algorithm.
3. wrap up your algorithm into a function that takes the instance as its first input.

## For loop

In for loops, ` in ...` where `...` must be an iterable -- check if it has `__iter__` method.

#### 1 Range

- `range(n)` is an iterable that generates a sequence of integers from 0 to n-1. 

In [42]:
a =range(10)
iter_a = a.__iter__() # obtain the iterator of a

An iterator is an object that has `__next__` method. `__next__` method returns the next element in the sequence. If there is no next element, it raises `StopIteration` exception.

In [43]:
iter_a.__next__()

0

- It can `__next__()` from 0 to 9.
- It means that `for i in range(10)` will have i take values from 0 to 9.

> `for i in  ...` basically run `i = ... .__next__()` for each loop until `StopIteration` exception is raised.
>
> So in our example, `simulate_onePeriod(ar)` will be called 10 times, and each time `i` will take values from 0 to 9.

In general a `for loop` can be expressed as:

```
for iterate(s) in iterable:
    do something
```

- `iterate(s)` can be one or more variables. (`i` in our example)
- `iterable` is an iterable object. (`range(10)` in our example)
- `do something` is the code that will be executed for each iteration under different values of `iterate(s)`. (`simulate_onePeriod(ar)` in our example)
- Each time the value of `iterate(s)` will be assigned as `iterate(s) = iterator.__next__()` where `iterator` comes from `iterable.__iter__()`. (`iterator = range(10).__iter__()` and `i = iterator.__next__()` in our example)

#### 2 list

In [44]:
a = ["apple", "banana", "orange"]
iter_a = a.__iter__() # obtain the iterator of a

In [ ]:
iter_a.__next__()

In [50]:
for i in a:
    print(i)

apple
banana
orange


- iterate `i`
- iterable `a`
- do something `print(i)`
- each time `i=iterator.__next__()` where `iterator = a.__iter__()`

#### 3 dictionary

In [53]:
a = {"name": "John", "age": 36, "country": "Norway"}
iter_a = a.__iter__() # obtain the iterator of a

In [ ]:
iter_a.__next__()

In [58]:
for key in a:
    print(a[key])

John
36
Norway


- iterate `k`
- iterable `a`
- do something `print(a[k])`
- each time `k=iterator.__next__()` where `iterator = a.__iter__()`

- dictionary has an `items()` method that returns an iterator that generates key-value pairs as tuples.

In [60]:
a_items = a.items()
a_items_iter = a_items.__iter__()

In [ ]:
a_items_iter.__next__()

In [65]:
for key, value in a.items():
    print(f'{key} is {value}')

name is John
age is 36
country is Norway


- iterates `key, value`
- iterable `a.items()`
- do something `print(f'{key} is {value}')`
- each time `key, value = iterator.__next__()` where `iterator = a.items().__iter__()`. Here you can see that `iterator.__next__()` will return a tuple of two values representing a key-value pair. And `key, value` will be assigned to the two values in the tuple respectively -- a technique called **unpacking**.


##### unpacking

In [67]:
# unpacking
k, v = ("name", "John")
print(k)
print(v)

name
John


> Instance as its FIRST input for later possible use as a method.

## Modulize your code

Under you .ipynb location create `py/AR/univariate.py` file and copy the following code into it:


In [184]:
import numpy as np

class AR:
    def __init__(self, *args, epsilon, Y0):
        self.phi = np.array(args)
        self.epsilon = epsilon
        self.Y0 = np.array(Y0)
        self.memory = np.array([])
        self.YPast = np.array(self.Y0)
    def simulate_nPeriods(self, n=1):
        simulate_nPeriods(self, n)
    def clear_memory(self):
        self.memory = np.array([])

# helpers

def simulate_onePeriod(ar, eps):
    # Simulate one period of ar process
    y_onePeriod_ahead = ar.phi@ar.YPast + eps

    # Update YPast
    ar.YPast = np.append(y_onePeriod_ahead, ar.YPast[:-1])

    ## append the new element to the memory
    ar.memory = np.append(ar.memory, y_onePeriod_ahead)

def simulate_nPeriods(ar, n):
    eps = ar.epsilon(n)
    for i in range(n):
        simulate_onePeriod(ar, eps[i])



> Helper functions can be placed at the back of the file. This is because of the **lazy evaluation** property of functions. Functions are not evaluated until they are called. 
>
> In other words, the order of function definition does not matter.

In [1]:
# import AR module
from py.TimeSeries.ar import AR
import numpy as np

def Epsilon(mu, sigma, seed=None):
    def draw(size):
        if seed:
            np.random.seed(seed)
        return np.random.normal(mu, sigma, size)
    return draw
epsilon = Epsilon(0, 0.4)
epsilon_withSeed = Epsilon(0, 0.4, seed=2023)


In [46]:
'without seed', epsilon(1), epsilon(10)

('without seed',
 array([-0.24029064]),
 array([-0.60789937,  0.83893168,  0.35702478,  0.63859508, -0.25550482,
        -0.4532189 ,  0.26263878,  0.2383147 , -0.32444786, -0.12021387]))

In [43]:
'with seed', epsilon_withSeed(1), epsilon_withSeed(10)

('with seed',
 array([0.28466941]),
 array([ 0.28466941, -0.12979398, -0.40074826,  0.09450032, -0.04086394,
        -0.45651705,  1.0617629 ,  0.57624208,  0.03956091, -1.24861286]))

> In most application, you should set seed for reproducible results.

In [29]:
ar = AR(0.8, -0.35, epsilon=epsilon, Y0=[0,0])
ar.simulate_nPeriods(10)
ar.memory

array([ 0.29142178,  0.77906526,  1.0583321 , -0.00869052,  0.0103743 ,
        0.02595584,  0.98986228,  0.95184478,  1.15259267,  0.50030369])

In [49]:
ar_withSeed = AR(0.8, -0.35, epsilon=epsilon_withSeed, Y0=[0,0])
ar_withSeed.simulate_nPeriods(10)
ar_withSeed.memory

array([ 0.28466941,  0.09794155, -0.42202931, -0.27740267, -0.11507582,
       -0.45148677,  0.74085003,  1.32694247,  0.84181737, -1.03958883])

- `from py.TimeSeries.ar import AR` look for `py/TimeSeries.ar.py` file from your current working directory. If you encounter `ModuleNotFoundError`, you can check your current working directory by `os.getcwd()` and make sure you are in the right directory.

In [47]:
# check current working directory
import os
os.getcwd()

'/Users/martin/Documents/GitHub/112-2-programming-for-economic-modeling/ipynb'

> You can set you current working directory by `os.chdir("path/to/your/directory")` so that `py/TimeSeries/ar.py` can be found under new current working directory.

In [ ]:
# graph the memory using plotly with x starts from 1
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, len(ar_withSeed.memory)+1), y=ar.memory))


![ar2_10](img/ar2_10.png)